- pip install jupyter
- python -m pip install spacy == 2.3.5
- pip install dill
- pip install pandas
- pip install nltk
- pip install transformers[sentencepiece]
- python -m spacy download en_core_web_sm, en_core_web_md, en_core_web_lg
- pip install torch
- pip install sklearn
- pip install lemminflect==0.2.1
- pip install -U sentence-transformers
- pip install fuzzywuzzy
- pip install python-Levenshtein

https://github.com/mmxgn/spacy-clausie
- pip install stanza

!git clone https://github.com/huggingface/neuralcoref.git
!pip install -U spacy
!python -m spacy download en

cd neuralcoref

!pip install -r requirements.txt
!pip install -e .

!pip freeze >> requirements.txt

import time
start = time.time()

In [1]:
%%time
from whqg.whqg_main import *

Wall time: 45.9 s


In [2]:
%%time
generated_questions = get_wh_questions("data/a1.txt", 100)

Wall time: 3min 54s


In [3]:
len(generated_questions)

100

In [4]:
for question in generated_questions:
    print(question)

Where regarded  most commonly as the period from the Third Dynasty through to the Sixth Dynasty (2686–2181 BC)  ?
During what era did the king of Egypt become a living god?
During what dynasty were pyramid complexes fewer devoted to construction than during the 4th?
What is the period in the third millennium also known as the 'Age of the Pyramids'?
What is  the revolutionary change in architecture accompanied by the effects on Egyptian society and economy of large-scale building projects ?
During what dynasty did the power of pharaoh gradually wane?
What is the period from the Third Dynasty through to the Sixth Dynasty?
What is the name of the period that was not recognized by Ancient Egyptians?
What is the name of the third period of civilization in Egypt?
What is the name of the country that was influenced by Egypt?
What been proposed  to be the work of Khafra and Khufu himself ?
The last king of the Early Dynastic Period was related to the first two kings of what kingdom?
Who been d

In [ ]:
with open("a1_self_gen_gsts.dill", "wb") as dill_file:
    pickle.dump(generated_questions, dill_file)

with open("../QA/questio-ns2", "wb") as dill_file:
    pickle.dump(generated_questions, dill_file)

f = open("a1.txt", encoding = "utf-8", mode = "r")
txt = f.read()
txt = txt[txt.find("\n\n\n")+3:txt.find("References")]

doc = nlp(txt)
ents = dict()
for ent in doc.ents:
    ents[ent.text] = ent.label

ents

contexts = []
questions = []
answers = []

def generate(sentences, ents):
    nlp3 = spacy.load('en_core_web_lg')
    claucy.add_to_pipe(nlp3)
    i = 0
    for sentence in sentences:
        doc = nlp3(sentence) 
        breakdown = doc._.clauses
        if len(breakdown) == 0:
            continue
        if (i != 0 and i != len(sentences) - 1):
            context = sentences[i - 1] + sentences[i] + sentences[i + 1]
        elif (i == 0):
             context = sentences[i] + sentences[i+1] + sentences[i+2]
        else:
             context = sentences[i-2] + sentences[i-1] + sentences[i]
            
        i+=1
        ie_data = pd.DataFrame(breakdown)[0].apply(string).str.split("~", expand = True).rename(columns={0:'type',
                                                                                              1:'subject',
                                                                                              2:'verb',
                                                                                              3:'indirect_obj',
                                                                                              4:'direct_obj',
                                                                                              5:'complement',
                                                                                              6:'adverbials'})
        ie_data = ie_data.replace("None", "")

        for row in ie_data.iterrows():
            if "NN" not in return_tag(row[1][1]) or row[1][2] == "":
                continue
            if ents.get(row[1][1]) in ["NORP","PERSON"]:
                tag = "Who "
            elif ents.get(row[1][1]) in ["DATE", "TIME"]:
                tag = "When "       
            else:
                tag = "What "

            end = row[1][4]
            if end == "":
                end = row[1][5]
            if end == "":
                end = row[1][6]
            if end == "":
                continue

            q = tag + row[1][2] + " " + row[1][3] + " " + end + " ?"
            keys = set(ents.keys())
            #_, e, _, _ = get_ents(q)
            for word in word_tokenize(q):
                if word in keys:
                    #q = paraphrase(q)
                    questions.append(q)
                    d = dict()
                    d["text"] = row[1][1]
                    d["answer_start"] = context.find(d["text"])
                    answers.append(d)
                    contexts.append(str(context))
                    break

generate(sent_tokenize(txt), ents)

In [ ]:
len(contexts), len(questions), len(answers)

for i in range(39):
    print("Context: ", contexts[i])
    print("Question: ", questions[i])
    print("Answer: ", answers[i])
    print()

from tqdm import tqdm

import os
for file in tqdm(os.listdir("../dataset")):
    if file.endswith(".txt"):
        print(os.path.join("../dataset", file))
        f = open(os.path.join("../dataset", file), encoding = "utf-8", mode = "r")
        text = f.read()
        sentences = sent_tokenize(text)
        doc = nlp(txt)
        entities = dict()
        for ent in doc.ents:
            entities[ent.text] = ent.label
        generate(sentences, entities)

len(contexts), len(questions), len(answers)

with open("combined", "rb") as dill_file:
    c = pickle.load(dill_file)

cc = c[:10200]

qq = c[10201:20401]

aa = c[20401:40600]

len(cc + contexts), len(questions + qq), len(answers+aa)

ccc = cc + contexts
qqq = questions + qq
aaa = aa + answers

len(ccc), len(qqq), len(aaa)

with open("sq_questions", "wb") as dill_file:
    pickle.dump(qqq, dill_file)
    
with open("sq_contexts", "wb") as dill_file:
    pickle.dump(ccc, dill_file)
    
with open("sq_answers", "wb") as dill_file:
    pickle.dump(aaa, dill_file)

from transformers import AutoTokenizer, AutoModelWithLMHead

tokenizer = AutoTokenizer.from_pretrained("flexudy/t5-base-multi-sentence-doctor")
model = AutoModelWithLMHead.from_pretrained("flexudy/t5-base-multi-sentence-doctor")


input_text = "repair_sentence: m a medical doct context: {That is my job I a}{or I save lives} </s>"
input_ids = tokenizer.encode(input_text, return_tensors="pt")
outputs = model.generate(input_ids, max_length=32, num_beams=1)
sentence = tokenizer.decode(outputs[0], skip_special_tokens=True, clean_up_tokenization_spaces=True)

input_text = "repair_sentence: What was  the first to gain some meaningful value using his observations made in 1832–1833 and South African astronomer Thomas Maclear's observations made in 1836–1837 ? context: {}{} </s>"
input_ids = tokenizer.encode(input_text, return_tensors="pt")
outputs = model.generate(input_ids, max_length=64, num_beams=1)
sentence = tokenizer.decode(outputs[0], skip_special_tokens=True, clean_up_tokenization_spaces=True)

end = time.time()
end - start

{u'1': [u'CARDINAL', u'DATE', u'EVENT', u'FAC', u'GPE', u'LANGUAGE', u'LAW', u'LOC', u'MONEY', u'NORP', u'ORDINAL', u'ORG', u'PERCENT', u'PERSON', u'PRODUCT', u'QUANTITY', u'TIME', u'WORK_OF_ART']}

str(breakdown[0].type) + str(breakdown[0].subject) + str(breakdown[0].verb) + str(breakdown[0].indirect_object) \
+ str(breakdown[0].direct_object) + str(breakdown[0].complement) + " ".join(breakdown[0].adverbials)

self.type, self.subject, self.verb, self.indirect_object, self.direct_object, self.complement,self.adverbials
{'SV', 'SVA', 'SVC', 'SVO', 'SVOC'}